## Part 1: Online Retail Market Basket Analysis

In [1]:
import pandas as pd
pd.set_option("max_colwidth", 150)

# Loading the online data
f = "https://github.com/cs6220/cs6220.spring2019/raw/master/data/Online%20Retail.xlsx"
df = pd.read_excel(f)
basket = (df[df["Country"] == "United Kingdom"]
          .groupby(["InvoiceNo", "Description"])["Quantity"]
          .sum().unstack().reset_index().fillna(0)
          .set_index("InvoiceNo")) # transform transactions into baskets of items
basket_sets = basket.applymap(lambda x: 1 if x >=1 else 0) # convert counts to booleans

In [2]:
basket_sets.head()

Description,20713,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1.1 - Frequent Itemset Generation

In [3]:
from mlxtend.frequent_patterns import apriori

In [4]:
support_threshold = 0.02 # Memory Error if below this value 

frequent_itemsets = apriori(basket_sets, min_support=support_threshold, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [5]:
# Top 5 1-itemsets with the highest support
frequent_itemsets[(frequent_itemsets['length'] == 1)].sort_values(by=['support'],ascending=False).tail()

,support,itemsets,length
181,0.020327,(TOY TIDY PINK POLKADOT),1
170,0.020327,(SMALL HEART MEASURING SPOONS),1
40,0.020327,(FELTCRAFT CUSHION OWL),1
21,0.020191,(CHOCOLATE THIS WAY METAL SIGN),1
114,0.020009,(PACK OF 60 SPACEBOY CAKE CASES),1


In [6]:
# Top 5 2-itemsets with the highest support
frequent_itemsets[(frequent_itemsets['length'] == 2)].sort_values(by=['support'],ascending=False).head()

,support,itemsets,length
211,0.035617,"(JUMBO BAG PINK POLKADOT, JUMBO BAG RED RETROSPOT)",2
207,0.031806,"(GREEN REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER )",2
218,0.031670,"(JUMBO STORAGE BAG SUKI, JUMBO BAG RED RETROSPOT)",2
217,0.029809,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG RED RETROSPOT)",2
223,0.027541,"(LUNCH BAG RED RETROSPOT, LUNCH BAG BLACK SKULL.)",2


In [7]:
# The highest support value for the 1-itemsets
frequent_itemsets[(frequent_itemsets['length'] == 1)].sort_values(by=['support'],ascending=False).head(1)['support']

188    0.098276
Name: support, dtype: float64

In [8]:
# The highest support value for the 2-itemsets
frequent_itemsets[(frequent_itemsets['length'] == 2)].sort_values(by=['support'], ascending=False).head(1)['support']

211    0.035617
Name: support, dtype: float64

### 1.2 - Association Rule Generation

In [9]:
from mlxtend.frequent_patterns import association_rules

In [10]:
confidence_threshold = 0.7

association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence_threshold).sort_values(by=['confidence'], ascending=False).head()[['antecedents','consequents','support','confidence','lift']]

,antecedents,consequents,support,confidence,lift
8,"(ROSES REGENCY TEACUP AND SAUCER , PINK REGENCY TEACUP AND SAUCER)",(GREEN REGENCY TEACUP AND SAUCER),0.022368,0.902930,21.306837
6,"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY TEACUP AND SAUCER)",(ROSES REGENCY TEACUP AND SAUCER ),0.022368,0.854419,19.677538
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.026180,0.820768,19.368019
5,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.024773,0.776671,17.886978
3,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.031806,0.750535,17.285056


The **different color variety of recency teacup and saucers** makes up the top association rules, which is very likely to be bought together since they could be complimentary products (same products but with different color options).

---
## Part 2: Association Rule Mining U.S. Census Data

In [11]:
import numpy as np
import pandas as pd

path = "https://raw.githubusercontent.com/cs6220/cs6220.spring2019/master/data/adult/"
names = pd.read_csv(path + "adult.names", sep="\n", header=None)
parse_cols = lambda x: x.str.split(":", expand=True).iloc[:, 0]
columns = np.roll(parse_cols(names.iloc[92:108, 0]), shift=-1)

df_adult = pd.read_csv(path + "adult.data", sep=",", header=None, index_col=False)
df_adult.columns = columns

In [12]:
from ipywidgets import interact, IntSlider
from IPython.display import display

# Credit source to: https://stackoverflow.com/questions/28778668/freeze-header-in-pandas-dataframe
def freeze_header(df, num_rows=30, num_columns=10, step_rows=1,
                  step_columns=1):
    """
    Freeze the headers (column and index names) of a Pandas DataFrame. A widget
    enables to slide through the rows and columns.
    """
    @interact(last_row=IntSlider(min=min(num_rows, df.shape[0]),
                                 max=df.shape[0],
                                 step=step_rows,
                                 description='rows',
                                 readout=False,
                                 disabled=False,
                                 continuous_update=True,
                                 orientation='horizontal',
                                 slider_color='purple'),
              last_column=IntSlider(min=min(num_columns, df.shape[1]),
                                    max=df.shape[1],
                                    step=step_columns,
                                    description='columns',
                                    readout=False,
                                    disabled=False,
                                    continuous_update=True,
                                    orientation='horizontal',
                                    slider_color='purple'))
    def _freeze_header(last_row, last_column):
        display(df.iloc[max(0, last_row-num_rows):last_row,
                        max(0, last_column-num_columns):last_column])

### 2.1 - Association Rule Mining

In [13]:
# df_adult.head()

#### One-Hot Encoding

In [14]:
# dropping all continuous columns
df_adult_nom = df_adult.select_dtypes([np.object])
df_adult_nom.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,">50K, <=50K."
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [15]:
#from sklearn.preprocessing import OneHotEncoder
#cat_encoder = OneHotEncoder()

In [16]:
# one-hot encoding the remaining columns
#df_adult_1hot = cat_encoder.fit_transform(df_adult_nom)
#df_adult_1hot;

In [17]:
#df_adult_1hot.toarray()

In [18]:
#cat_encoder.categories_

In [19]:
# one-hot encoding the remaining columns
from pandas import get_dummies
df_adult_1hot = pd.get_dummies(df_adult_nom)

### Rule Generation with Support = 0.3

In [20]:
support_threshold = 0.3

# Using support = 0.3
frequent_itemsets_1 = apriori(df_adult_1hot, min_support=support_threshold, use_colnames=True, max_len=3)
frequent_itemsets_1['length'] = frequent_itemsets_1['itemsets'].apply(lambda x: len(x))

#### Rule Interestingness with Confidence

In [21]:
confidence_threshold=0.7

rules_conf_1 = association_rules(frequent_itemsets_1, metric="confidence", min_threshold=confidence_threshold)
#rules_conf.head()

In [22]:
# Compute a column for antecedent length
rules_conf_1["antecedent_len"] = rules_conf_1["antecedents"].apply(lambda x: len(x))

In [23]:
# A quick look at the top rules using confidence as interestingness and ranked by confidence
freeze_header(num_rows=5, df=rules_conf_1[(rules_conf_1['confidence'] > 0.75) ].sort_values(by=['confidence'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']])

interactive(children=(IntSlider(value=5, description='rows', max=69, min=5, readout=False), IntSlider(value=6,…

In [24]:
rules_conf_1[(rules_conf_1['confidence'] > 0.75) ].sort_values(by=['confidence'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].iloc[[49]]

,antecedents,consequents,support,confidence,lift,antecedent_len
14,(native-country_ United-States),(race_ White),0.786862,0.878334,1.028165,1


I find the above rule interesting because it has quite a high support and confidence, meaning that the data has a high percentage of White Americans.

In [25]:
rules_conf_1[(rules_conf_1['confidence'] > 0.75) ].sort_values(by=['confidence'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].iloc[[0,1,4,7]]

,antecedents,consequents,support,confidence,lift,antecedent_len
11,(relationship_ Husband),(sex_ Male),0.405147,0.999924,1.494196,1
39,"(marital-status_ Married-civ-spouse, relationship_ Husband)",(sex_ Male),0.404871,0.999924,1.494196,2
5,(relationship_ Husband),(marital-status_ Married-civ-spouse),0.404902,0.999318,2.172729,1
42,(relationship_ Husband),"(marital-status_ Married-civ-spouse, sex_ Male)",0.404871,0.999242,2.442850,1


The above 4 rules are interesting in that they are the top rules ranked by confidence, but they are essentially facts with not much values if you are trying to gain useful information from the top rules. Also, notice how rule #40 and #43 are composed of same items but with different combinations in the antecedents and consequents. Interestingly #43 has a higher lift than that of #40. I think this is a good example of where rule A->B and B->A are considered different rules.

#### Rule Interestingness with Lift

In [26]:
lift_threshold = 1.2

rules_lift_1 = association_rules(frequent_itemsets_1, metric="lift", min_threshold=lift_threshold)

In [27]:
# Compute a column for antecedent length
rules_lift_1["antecedent_len"] = rules_lift_1["antecedents"].apply(lambda x: len(x))

In [28]:
# A quick look at the top rules using lift as interestingness and ranked by lift
freeze_header(num_rows=5, df=rules_lift_1[(rules_lift_1['lift'] > 0.85) ].sort_values(by=['lift'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']])

interactive(children=(IntSlider(value=5, description='rows', max=38, min=5, readout=False), IntSlider(value=6,…

In [29]:
rules_lift_1[(rules_lift_1['lift'] > 0.85) ].sort_values(by=['lift'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].iloc[[-1,-2]]

,antecedents,consequents,support,confidence,lift,antecedent_len
4,(marital-status_ Never-married),"(>50K, <=50K._ <=50K)",0.313012,0.954039,1.256653,1
5,"(>50K, <=50K._ <=50K)",(marital-status_ Never-married),0.313012,0.412298,1.256653,1


The above 2 rules are interetsing in that both points to the association between being never married to the income level. From the confidence level, we can see that, out of those who were never married, the majority were earning less than 50K; on the other hand, out of those earning less than 50K, only 41% were never married. This makes sense because a majority of the U.S. population earned less than 50K in 1996, whether married or unmarried.

In [30]:
rules_lift_1[(rules_lift_1['lift'] > 0.85) ].sort_values(by=['lift'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].iloc[[22,23]]

,antecedents,consequents,support,confidence,lift,antecedent_len
33,(sex_ Male),"(race_ White, relationship_ Husband)",0.366666,0.547912,1.494184,1
30,"(race_ White, relationship_ Husband)",(sex_ Male),0.366666,0.999916,1.494184,2


These 2 rules are interesting for the same reason as the previous 2 rules. It's another good example where rule A->B and B->A are considered different because the implications behind the 2 statements are very different.

In [31]:
rules_lift_1[(rules_lift_1['lift'] > 0.85) ].sort_values(by=['lift'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].iloc[[0,1]]

,antecedents,consequents,support,confidence,lift,antecedent_len
15,(relationship_ Husband),"(marital-status_ Married-civ-spouse, sex_ Male)",0.404871,0.999242,2.44285,1
14,"(marital-status_ Married-civ-spouse, sex_ Male)",(relationship_ Husband),0.404871,0.989789,2.44285,2


These 2 top rules are interesting in that they are almost identical in not only the components in the itemsets but also the support, confidence, and lift. Again, this is probably because they are rules of fact (i.e. by definition, a married male is called a husband, and a husband is a married male).

### Rule Generation with Support = 0.6

In [32]:
support_threshold = 0.6

# Using support = 0.6
frequent_itemsets_2 = apriori(df_adult_1hot, min_support=support_threshold, use_colnames=True, max_len=3)
frequent_itemsets_2['length'] = frequent_itemsets_2['itemsets'].apply(lambda x: len(x))

#### Rule Interestingness with Confidence

In [33]:
confidence_threshold=0.7

rules_conf_2 = association_rules(frequent_itemsets_2, metric="confidence", min_threshold=confidence_threshold)
#rules_conf.head()

In [34]:
# Compute a column for antecedent length
rules_conf_2["antecedent_len"] = rules_conf_2["antecedents"].apply(lambda x: len(x))

In [35]:
# A quick look at the top rules using confidence as interestingness and ranked by confidence
freeze_header(num_rows=5, df=rules_conf_2[(rules_conf_2['confidence'] > 0.75) ].sort_values(by=['confidence'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']])

interactive(children=(IntSlider(value=5, description='rows', max=6, min=5, readout=False), IntSlider(value=6, …

In [36]:
lift_threshold = 0.5

rules_lift_2 = association_rules(frequent_itemsets_2, metric="lift", min_threshold=lift_threshold)

In [37]:
# Compute a column for antecedent length
rules_lift_2["antecedent_len"] = rules_lift_2["antecedents"].apply(lambda x: len(x))

In [38]:
# A quick look at the top rules using lift as interestingness and ranked by lift
freeze_header(num_rows=5, df=rules_lift_2.sort_values(by=['lift'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']])

interactive(children=(IntSlider(value=5, description='rows', max=8, min=5, readout=False), IntSlider(value=6, …

### Rule Comparison

Top rules with support level = 0.3

In [39]:
rules_conf_1.sort_values(by=['confidence'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].head()

,antecedents,consequents,support,confidence,lift,antecedent_len
11,(relationship_ Husband),(sex_ Male),0.405147,0.999924,1.494196,1
39,"(marital-status_ Married-civ-spouse, relationship_ Husband)",(sex_ Male),0.404871,0.999924,1.494196,2
58,"(race_ White, relationship_ Husband)",(sex_ Male),0.366666,0.999916,1.494184,2
64,"(native-country_ United-States, relationship_ Husband)",(sex_ Male),0.364239,0.999916,1.494183,2
5,(relationship_ Husband),(marital-status_ Married-civ-spouse),0.404902,0.999318,2.172729,1


In [40]:
rules_lift_1.sort_values(by=['lift'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].head()

,antecedents,consequents,support,confidence,lift,antecedent_len
15,(relationship_ Husband),"(marital-status_ Married-civ-spouse, sex_ Male)",0.404871,0.999242,2.442850,1
14,"(marital-status_ Married-civ-spouse, sex_ Male)",(relationship_ Husband),0.404871,0.989789,2.442850,2
8,"(race_ White, marital-status_ Married-civ-spouse)",(relationship_ Husband),0.366420,0.889709,2.195848,2
11,(relationship_ Husband),"(race_ White, marital-status_ Married-civ-spouse)",0.366420,0.904343,2.195848,1
21,(relationship_ Husband),"(native-country_ United-States, marital-status_ Married-civ-spouse)",0.363994,0.898355,2.188162,1


Top rules with support Level = 0.6

In [41]:
rules_conf_2.sort_values(by=['confidence'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].head()

,antecedents,consequents,support,confidence,lift,antecedent_len
2,(race_ White),(native-country_ United-States),0.786862,0.921089,1.028165,1
5,"(>50K, <=50K._ <=50K)",(native-country_ United-States),0.675624,0.889927,0.993381,1
0,(workclass_ Private),(native-country_ United-States),0.618378,0.887161,0.990293,1
1,(native-country_ United-States),(race_ White),0.786862,0.878334,1.028165,1
3,"(>50K, <=50K._ <=50K)",(race_ White),0.635699,0.837338,0.980176,1


In [42]:
rules_lift_2.sort_values(by=['lift'], ascending=False)[['antecedents','consequents','support','confidence','lift','antecedent_len']].head()

,antecedents,consequents,support,confidence,lift,antecedent_len
2,(native-country_ United-States),(race_ White),0.786862,0.878334,1.028165,1
3,(race_ White),(native-country_ United-States),0.786862,0.921089,1.028165,1
6,"(>50K, <=50K._ <=50K)",(native-country_ United-States),0.675624,0.889927,0.993381,1
7,(native-country_ United-States),"(>50K, <=50K._ <=50K)",0.675624,0.754165,0.993381,1
0,(native-country_ United-States),(workclass_ Private),0.618378,0.690264,0.990293,1


The comparison above shows that, **at a low level of support (e.g. support = 0.3), the top rules using the two different interestingness measures can be very different**. Although they all have the value "relationship_Husband" in the itemsets, 4 of the top 5 rules using confidence as the interestingness measure have "sex_Male" as the consequents. The rules using lift seems more of a mix with "relationship_Husband" in antecedents and consequents. This might be because **when the support threshold is set to only 0.3, there's a bigger basket of itemsets that meets this criteria, so we would end up with more associations rules**.

On the other hand, **when the level of support is higher (e.g. support = 0.6), 3 of the top 5 rules using different interestingness measures are the same**. My reasoning would be that **with support level at 0.6, only a limited number of itemsets can be found** (see the cell below). Thus, the association rules that can be generated are very limited as well, which leads to the high portion of intersection between the top rules using different interestingness metrics.

In [43]:
# All frequent itemsets with minimum support threshold set to 0.3
freeze_header(num_rows=8,df=frequent_itemsets_1)

interactive(children=(IntSlider(value=8, description='rows', max=47, min=8, readout=False), IntSlider(value=3,…

In [44]:
# All frequent itemsets with minimum support threshold set to 0.6
frequent_itemsets_2

,support,itemsets,length
0,0.697030,(workclass_ Private),1
1,0.854274,(race_ White),1
2,0.669205,(sex_ Male),1
3,0.895857,(native-country_ United-States),1
4,0.759190,"(>50K, <=50K._ <=50K)",1
5,0.618378,"(native-country_ United-States, workclass_ Private)",2
6,0.786862,"(native-country_ United-States, race_ White)",2
7,0.635699,"(>50K, <=50K._ <=50K, race_ White)",2
8,0.675624,"(>50K, <=50K._ <=50K, native-country_ United-States)",2
